# TP - HMM
## Import


In [1]:
import numpy as np

## Variables

In [2]:
states = ("normal","confinement",'vacances')

obs =("sport","courses","sport","dormir")

emit_p = {
    'normal' :  {'dormir' : 0.4, 'sport' : 0.3, 'courses' : 0.3},
    'confinement' :  {'dormir' : 0.4, 'sport' : 0.2, 'courses' : 0.4},
    'vacances' : {'dormir' : 0.7, 'sport' : 0.1, 'courses' : 0.2}
}

start_p = {
    'normal' : 0.6,
    'confinement' : 0.1,
    'vacances' : 0.3
}

trans_p = {
    'normal' : {'normal' : 0.8, 'confinement' : 0.1, 'vacances' : 0.1},
    'confinement' : {'normal' : 0.3, 'confinement' : 0.6, 'vacances' : 0.1},
    'vacances' : {'normal' : 0.5, 'confinement' : 0.1, 'vacances' : 0.4}
}

## Algorithme

In [3]:
def trouve_chemin(mat_res) :
    opt = []
    max_prob = 0.0
    previous = None
    
    # Trouve la proba max a la derniere etape de la suite d'action
    for st, data in mat_res[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Part de la fin et remonte jusque l'element 0 avec la proba max
    # ajoute l'etat precedent de la plus grande proba à l'indice 0 de opt
    for col in range(len(mat_res) - 2, -1, -1):
        previous = mat_res[col + 1][previous]["prev"]
        opt.insert(0, previous)
    return opt

In [4]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    # Initialise la matrice de retour
    mat_res = [{}]
    for st in states:
        mat_res[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}

    # pour chaque etape de la suite d'action observer
    for col in range(1, len(obs)):
        # ajout d'une colonne a mat_res
        mat_res.append({})

        # pour chacun des etats
        for etat in states:
            max_tr_prob = mat_res[col - 1][states[0]]["prob"] * trans_p[states[0]][etat]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = mat_res[col - 1][prev_st]["prob"] * trans_p[prev_st][etat]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[etat][obs[col]]
            mat_res[col][etat] = {"prob": max_prob, "prev": prev_st_selected}
    return mat_res, max_prob

## Exécution

In [5]:
mat,proba_max = viterbi(obs, states, start_p, trans_p, emit_p)

## Affichage

In [6]:
def affiche_table(mat):
    """
    Affiche la table de resultat de viterbi 
    """
    str_mat = ''
    for etat in mat[0]:
        str_mat +=  str(etat) + ' : '
        for ligne in mat:
            str_mat += ' %.4f' % ligne[etat]["prob"]
        str_mat += '\n'
    print(str_mat)

def affiche_chemin(mat, proba_max) : 
    opt = trouve_chemin(mat)
    print("Suite des differents états pour le meilleur resultat : ", proba_max)
    for i in opt:
        print(i)

In [7]:
affiche_table(mat)
affiche_chemin(mat, proba_max)

normal :  0.1800 0.0432 0.0104 0.0033
confinement :  0.0200 0.0072 0.0009 0.0004
vacances :  0.0300 0.0036 0.0004 0.0007

Suite des differents états pour le meilleur resultat :  0.00072576
normal
normal
normal
normal
